In [1]:
import os
import re
import json
import pickle

In [2]:
with open("../gptsmall_filtered_index.pkl", "rb") as f:
    index_list = pickle.load(f)

In [6]:
input_dir = './txt'
output_dir = './cq'

file_data = []
bad_filenames = []

In [7]:
import re

def clean_code(text, new_filename):
    start = text.find("import cadquery")
    if start == -1:
        print("can not find 'import cadquery'")
        return None

    cleaned = text[start:]

    export_pattern = r"cq\.exporters\.export\s*\(\s*([^,]+),\s*['\"].*?\.stl['\"].*?\)"
    matches = list(re.finditer(export_pattern, cleaned))

    incomplete_pattern = r"cq\.exporters\.[^\n]*$"
    incomplete_matches = re.findall(incomplete_pattern, cleaned, re.MULTILINE)
    
    if not matches and not incomplete_matches:
        print("[WARNING] No export statement found")
        return None

    first_param = matches[0].group(1).strip()
    
    cleaned_wo_exports = re.sub(export_pattern, "", cleaned)
    
    lines = cleaned_wo_exports.split('\n')
    cleaned_lines = []
    for line in lines:
        if not re.search(r"cq\.exporters\.", line):
            cleaned_lines.append(line)
    cleaned_wo_exports = '\n'.join(cleaned_lines)
    
    final_code = cleaned_wo_exports.strip() + f"\n\ncq.exporters.export({first_param}, \"{new_filename}\")"

    return final_code

In [8]:
failed_case = []

In [ ]:
for i in range(5198):
    full_path = os.path.join(input_dir, f'{i}.txt') 
    with open(full_path, "r", encoding="utf-8") as f:
        text = f.read()
        cleaned_code = clean_code(text, f'./cq/{index_list[i]}')
    if cleaned_code:
        filename = index_list[i].replace(".stl", "")
        output_path = os.path.join(output_dir, f"{filename}.py")
        with open(output_path, "w", encoding="utf-8") as f_out:
            f_out.write(cleaned_code)
    else:
        print(f'error: {i} no cleaned code')
        failed_case.append(i)

In [24]:
failed_files = []
success_files = []

In [ ]:
import os
import subprocess

python_dir = './cq'

for filename in sorted(os.listdir(python_dir)):
    if filename.endswith('.py') and not filename.startswith('._'):
        file_path = os.path.join(python_dir, filename)
        try:
            result = subprocess.run(['python', file_path], capture_output=True, text=True, timeout=30)
            
            if result.returncode != 0:
                print(f"FAILED] Execution failed: {filename}")
                failed_files.append(filename)
            else:
                print(f"[SUCCESS] Successfully executed: {filename}")
                success_files.append(filename)

        except subprocess.TimeoutExpired:
            print(f"[TIMEOUT] Execution timed out: {filename}")
            failed_files.append(filename)

        except Exception as e:
            print(f"ERROR] Exception occurred: {filename} -> {e}")
            failed_files.append(filename)


In [ ]:
print(len(failed_files))
print(len(success_files))